# Análise de suicídios

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

#### Leitura dos dados

In [2]:
df = pd.read_csv('master.csv')

Vamos dar uma olhada nos dados:

In [3]:
df.sample(5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
14159,Kiribati,2000,female,5-14 years,0,10475,0.00,Kiribati2000,NaN,"67,254,174",928,Millenials
12221,Ireland,1998,female,75+ years,3,111741,2.68,Ireland1998,NaN,"90,082,034,316",26115,G.I. Generation
9006,Finland,2009,male,15-24 years,89,336510,26.45,Finland2009,NaN,"251,499,027,508",49878,Millenials
17065,Montenegro,2009,female,5-14 years,0,39987,0.00,Montenegro2009,NaN,"4,159,330,370",7023,Generation Z
13909,Kazakhstan,2004,female,75+ years,61,271208,22.49,Kazakhstan2004,NaN,"43,151,647,003",3110,Silent


In [4]:
df.describe()

,year,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
count,27820.000000,27820.000000,2.782000e+04,27820.000000,8364.000000,27820.000000
mean,2001.258375,242.574407,1.844794e+06,12.816097,0.776601,16866.464414
std,8.469055,902.047917,3.911779e+06,18.961511,0.093367,18887.576472
min,1985.000000,0.000000,2.780000e+02,0.000000,0.483000,251.000000
25%,1995.000000,3.000000,9.749850e+04,0.920000,0.713000,3447.000000
50%,2002.000000,25.000000,4.301500e+05,5.990000,0.779000,9372.000000
75%,2008.000000,131.000000,1.486143e+06,16.620000,0.855000,24874.000000
max,2016.000000,22338.000000,4.380521e+07,224.970000,0.944000,126352.000000


O dataset tem dados de suicídios de 1985 até 2016.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
country               27820 non-null object
year                  27820 non-null int64
sex                   27820 non-null object
age                   27820 non-null object
suicides_no           27820 non-null int64
population            27820 non-null int64
suicides/100k pop     27820 non-null float64
country-year          27820 non-null object
HDI for year          8364 non-null float64
 gdp_for_year ($)     27820 non-null object
gdp_per_capita ($)    27820 non-null int64
generation            27820 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


Existem dados nulos?

In [6]:
df.isnull().sum()

country                   0
year                      0
sex                       0
age                       0
suicides_no               0
population                0
suicides/100k pop         0
country-year              0
HDI for year          19456
 gdp_for_year ($)         0
gdp_per_capita ($)        0
generation                0
dtype: int64

## Entendendo os dados

O campo country-year apresenta o nome do país e o ano do registro. Desta forma, é uma campo redundante e será descartado. Também devido a maioria dos dados do campo 'HDI for year', será descartado. 

In [7]:
df.drop(['country-year', 'HDI for year'], inplace=True, axis = 1)

Vamos renomear algumas colunas simplesmente para ficar mais fácil de acessá-las.

In [8]:
df = df.rename(columns={'gdp_per_capita ($)': 'gdp_per_capita', ' gdp_for_year ($) ':'gdp_for_year'})

Apenas para deixar claro, GDP é o PIB do país em questão. No caso, o campo 'gdp_for_year' está como uma string, então vamos converter isto para num número.

In [9]:
for i, x in enumerate(df['gdp_for_year']):
    df['gdp_for_year'][i] = float(x.replace(',', ''))

/home/luis/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Descrição dos dados

Cada dado no conjunto de dados representa um ano, de um país, uma determinada faixa de idade, e um sexo. Por exemplo, no país Brasil no ano de 1985, com mais de 75 anos, suicidaram 129 homens.

O conjunto de dados apresenta 10 atributos. Sendo estes:

- País: país do dados do registro;
- Ano: ano dos dados do registro;
- Sexo: Sexo (homem ou mulher);
- Idade: Faixa de idade dos suicidas, idades divididas em seis faixas;
- Suicides_no: quantidade de suicidas;
- População: população do país em questão neste ano;
- Suicides/100k pop: Razão entre a quantidade de suicidas e a população / 100k;
- GDP_for_year: PIP do país no ano quem questão;
- GDP_per_capita: razão entre o PIP do país e sua população;
- Generation: Geração dos suicidas em questão, sendo possíveis 6 diferentes categorias.

Possíveis faixas de idade e gerações são:

In [18]:
df['age'].unique()

array(['15-24 years', '35-54 years', '75+ years', '25-34 years',
       '55-74 years', '5-14 years'], dtype=object)

In [19]:
df['generation'].unique()

array(['Generation X', 'Silent', 'G.I. Generation', 'Boomers',
       'Millenials', 'Generation Z'], dtype=object)